In [5]:
import os, sys
sys.path.append(os.path.abspath('../src/bin/lib/')) # add path to lib
sys.path.append(os.path.abspath('../../schema/')) # add path nmdc.py

In [6]:
import yaml
import json
from yaml import CLoader as Loader, CDumper as Dumper
import data_operations as dop
from dotted_dict import DottedDict
from collections import namedtuple
import data_operations as dop
from data_operations import make_dataframe, make_dataframe_from_spec_file, unpivot_dataframe
import pandas as pds
import numpy as np

from pandasql import sqldf
def pysqldf(q):
    return sqldf(q, globals())

In [7]:
spec_file = "../src/bin/lib/nmdc_data_source.yaml"
with open(spec_file, 'r') as input_file:
    spec = DottedDict(yaml.load(input_file, Loader=Loader))

In [8]:
# mdf = make_dataframe_from_spec_file (spec_file, nrows=5)
# mdf.head()

In [9]:
# mdf['nmdc_record_id'] = mdf['nmdc_record_id'].astype(str)
# mdf.head()
# mdf.sort_values(by='nmdc_record_id')
# mdf.sort_values(by='value')
# mdf[mdf.nmdc_record_id == '114663']

In [10]:
# read data from saved file
mdf = pds.read_csv('../src/data/nmdc_merged_data.tsv.zip', sep='\t', dtype=str)
# mdf = pds.read_csv('../src/data/nmdc_merged_data.tsv.zip', sep='\t', dtype=str, nrows=100)

In [11]:
mdf.nmdc_data_source.unique()

array(['study_table', 'contact_table', 'proposals_table', 'project_table',
       'project_biosample_table', 'biosample_table', 'ficus_faa_table',
       'ficus_fna_table', 'ficus_fastq_table', 'ficus_jgi_emsl',
       'ficus_emsl'], dtype=object)

In [12]:
tagsdf = pds.DataFrame({
    'user_id':[1,2,1,3,3,3,4,5],
    'content_id':[1,1,2,2,2,2,3,4],
    'tag':['cool','nice','clever','clever','not-bad', '', 'foo', 'bar']})


In [13]:
tagsdf

,user_id,content_id,tag
0,1,1,cool
1,2,1,nice
2,1,2,clever
3,3,2,clever
4,3,2,not-bad
5,3,2,
6,4,3,foo
7,5,4,bar


In [14]:
# tagsdf.groupby("content_id").agg({'tag': lambda tag: ','.join(filter(None, tag))})

In [15]:
# tagsdf['tags'] = tagsdf.groupby('content_id')['tag'].apply(lambda tags: ','.join( filter(None, tags) ))
# tagsdf['tags'] = tagsdf.groupby('content_id')['tag'].apply(lambda tags: ','.join(tags))
# tagsdf['tags'] = tagsdf.groupby("content_id").agg({'tag': lambda tag: ','.join(tag)})
groups = pds.DataFrame(tagsdf.groupby('content_id')['tag'].apply(lambda tags: ','.join( filter(None, tags) ))).reset_index()
groups.rename(columns={'tag':'tags'}, inplace=True)

In [16]:
groups

,content_id,tags
0,1,"cool,nice"
1,2,"clever,clever,not-bad"
2,3,foo
3,4,bar


In [17]:
tagsdf = pds.merge(tagsdf, groups, how='inner', on='content_id')

In [18]:
tagsdf

,user_id,content_id,tag,tags
0,1,1,cool,"cool,nice"
1,2,1,nice,"cool,nice"
2,1,2,clever,"clever,clever,not-bad"
3,3,2,clever,"clever,clever,not-bad"
4,3,2,not-bad,"clever,clever,not-bad"
5,3,2,,"clever,clever,not-bad"
6,4,3,foo,foo
7,5,4,bar,bar


In [19]:
# tagsdf.dropna(subset=['tags'], inplace=True)

In [20]:
# tagsdf

In [21]:
tagsdf.drop(columns=['tag'], inplace=True)

In [22]:
tagsdf

,user_id,content_id,tags
0,1,1,"cool,nice"
1,2,1,"cool,nice"
2,1,2,"clever,clever,not-bad"
3,3,2,"clever,clever,not-bad"
4,3,2,"clever,clever,not-bad"
5,3,2,"clever,clever,not-bad"
6,4,3,foo
7,5,4,bar


In [23]:
tagsdf.drop_duplicates()

,user_id,content_id,tags
0,1,1,"cool,nice"
1,2,1,"cool,nice"
2,1,2,"clever,clever,not-bad"
3,3,2,"clever,clever,not-bad"
6,4,3,foo
7,5,4,bar


In [24]:
study_table = dop.extract_table(mdf, 'study_table')
jgi_emsl_table = dop.extract_table(mdf, 'ficus_jgi_emsl')
emsl_table = dop.extract_table(mdf, 'ficus_emsl')

In [25]:
q = """
select 
    emsl_table.*, jgi_emsl_table.gold_study_id
from
    emsl_table
inner join
    jgi_emsl_table
on
    emsl_table.emsl_proposal_id = jgi_emsl_table.emsl_proposal_id
"""
sqldf(q).head()

,nmdc_record_id,alkylation_state,autosampler_vialing_concentration,autosampler_vialing_volume,campaign_collaborators,campaign_comment,campaign_created,campaign_description,campaign_emsl_user_proposal_list,campaign_experiment_prefixes,...,sample_prep_request_prep_method,sample_prep_request_reason,sample_prep_request_requestor,sample_prep_request_sample_level_blocking_and_randomization,sample_prep_request_sample_naming_convention,sample_prep_request_sample_type,sample_prep_request_special_instructions,sample_prep_request_state,sample_prep_request_technical_replicates,gold_study_id
0,359123,Not Alkylated,None,100 uL,None,None,2001-02-14 16:31:46.920000,None,20496.0,None,...,None,None,None,None,None,None,None,None,None,Gs0110132
1,359124,Not Alkylated,None,100 uL,None,None,2001-02-14 16:31:46.920000,None,20496.0,None,...,None,None,None,None,None,None,None,None,None,Gs0110132
2,359129,Not Alkylated,None,100 uL,None,None,2001-02-14 16:31:46.920000,None,20496.0,None,...,None,None,None,None,None,None,None,None,None,Gs0110132
3,359130,Not Alkylated,None,100 uL,None,None,2001-02-14 16:31:46.920000,None,20496.0,None,...,None,None,None,None,None,None,None,None,None,Gs0110132
4,359131,Not Alkylated,None,100 uL,Dr. Matthias Hess,None,2014-01-17 11:18:48.967000,FECB: A Functional Encyclopedia of Cyanobacteria,48099.0,None,...,Chloroform/methanol extraction,Test methanol/chloroform extraction on cyanoba...,"Zink, Erika M",No,Hess_,Cell pellet,None,Closed,3.0,Gs0110132


In [26]:
study_table_splice = study_table[['study_id', 'gold_id']].copy()
jgi_emsl_table_splice = jgi_emsl_table[['gold_study_id', 'emsl_proposal_id']]

In [27]:
temp1_df = \
        pds.merge(jgi_emsl_table_splice, study_table_splice, how='inner', left_on='gold_study_id', right_on='gold_id')

In [28]:
temp1_df

,gold_study_id,emsl_proposal_id,study_id,gold_id
0,Gs0110115,48086,110115,Gs0110115
1,Gs0110132,48099,110132,Gs0110132
2,Gs0112340,48467,112340,Gs0112340
3,Gs0114298,48484,114298,Gs0114298
4,Gs0114663,48473,114663,Gs0114663
5,Gs0114675,48483,114675,Gs0114675
6,Gs0120351,48462,120351,Gs0120351
7,Gs0128849,49521,128849,Gs0128849
8,Gs0130354,49483,130354,Gs0130354
9,Gs0133461,49001,133461,Gs0133461


In [29]:
study_table = dop.extract_table(mdf, 'study_table')
contact_table = dop.extract_table(mdf, 'contact_table')
proposals_table = dop.extract_table(mdf, 'proposals_table')
project_table = dop.extract_table(mdf, 'project_table')
jgi_emsl_table = dop.extract_table(mdf, 'ficus_jgi_emsl')
emsl_table = dop.extract_table(mdf, 'ficus_emsl')
faa_table = dop.extract_table(mdf, 'ficus_faa_table')
fna_table = dop.extract_table(mdf, 'ficus_fna_table')
fastq_table = dop.extract_table(mdf, 'ficus_fasq_table')
project_biosample_table = dop.extract_table(mdf, 'project_biosample_table')
biosample_table = dop.extract_table(mdf, 'biosample_table')

In [33]:
data_objects = dop.make_data_objects_dataframe(faa_table, fna_table, fastq_table, project_table)
data_objects_dictdf = data_objects.to_dict(orient="records") # transorm dataframe to dictionary

AttributeError: 'DataFrame' object has no attribute 'file_id'

In [ ]:
## build a dataframe with each project id and the file ids associated with it.
q = """
select
    d1.gold_project_id, group_concat(d2.file_id, ",") as file_ids, count(d2.file_id) as file_count
from
    data_objects d1
inner join
    data_objects d2
on
    d1.file_id = d2.file_id
group by 
    d1.gold_project_id
"""
files_df = sqldf(q)

In [ ]:
files_df

In [ ]:
pds.set_option('display.width', 10000)
pds.set_option('display.max_colwidth', 2000)
files_df.sort_values(by='gold_project_id')

In [ ]:
# groups = data_objects.groupby('gold_project_id')['file_id']
groups = data_objects.groupby('gold_project_id')
# pds.DataFrame(groups).sort_values(by=0)
# pds.DataFrame(groups)

In [ ]:
# for g in groups: print(type(g[1]))
for idx, g in enumerate(groups): 
    ser = g[1].file_id
    print(list(ser))
    if idx > 0: break
# for g in groups: print(g[1])
# groups = files_df.groupby('gold_project_id')['file_ids']
# pds.DataFrame(groups).sort_values(by=0)
# groups[1].apply(lambda x:','.join(filter(None, x))).reset_index()

In [ ]:
fgroups = data_objects.groupby('gold_project_id')['file_id']

In [ ]:
for idx, g in enumerate(fgroups): 
    ser = g[1]
    print(ser)
    print(list(ser))
    if idx > 0: break

In [ ]:
pds.DataFrame(fgroups.apply(lambda x:','.join(filter(None, x)))).drop_duplicates().reset_index()

In [ ]:
df = pds.DataFrame(fgroups.apply(lambda x: list(x)).reset_index())
df
# df.drop_duplicates(inplace=True)

In [ ]:
# files_df.head(None)
# files_df

In [ ]:
project = dop.make_project_dataframe (project_table, study_table, contact_table, data_objects)

In [ ]:
# project.output_files.unique()
project[['output_files']].head()

In [ ]:
groups = data_objects.groupby('gold_project_id')['file_id']

output_files = \
    pds.DataFrame(groups.apply(lambda x:','.join(filter(None, x)))).drop_duplicates().reset_index()
output_files.rename(columns={'file_id': 'output_files'}, inplace=True)

In [ ]:
output_files.head()

In [ ]:
# project.gold_id